## Calculate Pattern Similarity Maps

#### 1. How to get single subject correlation maps:

1. get unsmoothed univariate maps for other and other at subbrick pain vs no pain (20) or anticipation vs no anticipation (26)
2. get whole brain mask 
3. choose mni template
4. make searchlight of 3 mm x 3 mm
5. extract values similar to 3dmaskdump
6. calculate spearman correlation
7. take the arctanh to standardize


### **1. Searchlight using rsa toolbox**

https://rsatoolbox.readthedocs.io/en/stable/demo_searchlight.html#Searchlight-for-RSA

https://rsatoolbox.readthedocs.io/en/stable/



In [ ]:
### Four masks: wholebrain, dACC, leftAI, rightAI

import os
import glob
import numpy as np
import nibabel as nib
from rsatoolbox.util.searchlight import get_volume_searchlight, get_searchlight_RDMs
from nilearn.image import new_img_like
from nilearn.plotting import plot_stat_map
import matplotlib.pyplot as plt

# Paths
nifti_dir = '/my/path/fMRIData/1stlevel_glm/'  # Directory containing the unsmoothed stats files
wholebrain_mask_path = '/my/path/fMRIData/masks/MNI152NLin2009cAsym-desc-graymatter-thr25_3mm-mask_resampled.nii.gz'  # Whole-brain mask
dACC_mask_path = '/my/path/fMRIData/masks/aal2_desc-dACC-anteriorY0_3mm-mask_resampled.nii.gz'  # dACC mask
leftAI_mask_path = '/my/path/fMRIData/masks/aal2_desc-left-AI_3mm-mask_resampled.nii.gz'  # left AI mask
rightAI_mask_path = '/my/path/fMRIData/masks/aal2_desc-right-AI_3mm-mask_resampled.nii.gz'  # right AI mask
output_base_dir = '/my/path/fMRIData/mvpa/pattern_similarity/rsatoolbox/'  # Base output directory
template_path = '/my/path/fMRIData/masks/mni152_0-737mm_plotting.nii.gz'  # Template for plotting

# Ensure the base output directory exists
os.makedirs(output_base_dir, exist_ok=True)

# Parameters for conditions
conditions = {
    'pain': {'subbrick_index': 20, 'output_suffix': 'pain'},
    'anticipation': {'subbrick_index': 26, 'output_suffix': 'anticipation'}
}

# Searchlight Parameters
radius = 3  # Searchlight radius in voxels (9mm)
min_voxel_count_ratio = 0.3  # Minimum proportion of valid voxels required, changed it to 0.3 because that's the default for TDT in MATLAB

# Masks to iterate through
masks = {
    "wholebrain": wholebrain_mask_path,
    "dACC": dACC_mask_path,
    "leftAI": leftAI_mask_path,
    "rightAI": rightAI_mask_path

}

# Helper function: Extract and Fisher Z-transform the single off-diagonal value
def upper_tri_z_from_dissimilarity(RDM):
    if RDM.ndim == 1:  # If RDM is already flattened (1D array)
        return np.arctanh(1 - RDM[0])  # The single value is the off-diagonal element subtracted from 1
    elif RDM.shape[0] == 2:  # For a 2x2 RDM
        return np.arctanh(1 - RDM[0, 1])  # The upper triangle value is the off-diagonal element subtracted from 1
    else:
        raise ValueError("Unexpected RDM shape: RDM must be 1D or 2D with shape (2, 2).")

# List all subject directories
subject_dirs = [d for d in os.listdir(nifti_dir) if os.path.isdir(os.path.join(nifti_dir, d))]

# Process each condition and mask
for condition_name, params in conditions.items():
    print(f"Processing condition: {condition_name}")

    for mask_name, mask_path in masks.items():
        print(f"Using mask: {mask_name}")
        
        # Load the mask
        mask_img = nib.load(mask_path)
        mask_data = mask_img.get_fdata().astype(bool)

        # Identify searchlight centers and neighbors
        centers, neighbors = get_volume_searchlight(mask_data, radius, threshold=min_voxel_count_ratio)


        # Print the number of voxels in each searchlight
        for i, center in enumerate(centers):
            print(f"Searchlight center {i} at {center} has {len(neighbors[i])} voxels")

        # Create output subdirectories for the mask
        condition_output_dir = os.path.join(output_base_dir, condition_name, mask_name)
        condition_images_dir = os.path.join(condition_output_dir, "images")
        os.makedirs(condition_output_dir, exist_ok=True)
        os.makedirs(condition_images_dir, exist_ok=True)

        # Process all participants
        for subject_id in subject_dirs:
            print(f"Processing subject {subject_id}")

            # Find self and other stats files within the subject folder
            self_file = glob.glob(os.path.join(nifti_dir, subject_id, f'{subject_id}_task-empathy_glm-self_*unsmoothed_stats.nii.gz'))
            other_file = glob.glob(os.path.join(nifti_dir, subject_id, f'{subject_id}_task-empathy_glm-other_*unsmoothed_stats.nii.gz'))

            if not self_file or not other_file:
                print(f"Skipping {subject_id} as stats files are missing.")
                continue

            self_file = self_file[0]
            other_file = other_file[0]

            # Load the specified subbrick for this condition
            self_img = nib.load(self_file)
            other_img = nib.load(other_file)
            self_data = self_img.slicer[..., params['subbrick_index']].get_fdata()
            other_data = other_img.slicer[..., params['subbrick_index']].get_fdata()

            # Prepare data for rsatoolbox
            data_2d = np.stack([self_data, other_data], axis=0).reshape(2, -1)
            # Exclude NaN voxels
            valid_voxels = ~np.isnan(data_2d).any(axis=0)
            data_2d = data_2d[:, valid_voxels]

            # Compute Searchlight RDMs
            print(f"Starting searchlight RDM computation for {subject_id} ({condition_name}) using {mask_name} mask...")
            SL_RDM = get_searchlight_RDMs(data_2d, centers, neighbors, np.array([0, 1]), method='correlation')

            # Extract the single Fisher Z-transformed value from each RDM
            z_transformed_rdm = np.zeros(mask_data.shape)
            for idx, dissimilarity in enumerate(SL_RDM.dissimilarities):  # Access raw RDMs
                if dissimilarity is not None and dissimilarity.size > 0:
                    z_value = upper_tri_z_from_dissimilarity(dissimilarity)  # Apply Fisher's Z-transformation
                    voxel_idx = np.unravel_index(centers[idx], mask_data.shape)
                    z_transformed_rdm[voxel_idx] = z_value

            # Save the resulting map
            result_img = new_img_like(mask_img, z_transformed_rdm)
            output_path = os.path.join(condition_output_dir, f'{subject_id}_rsatoolbox_rdm_map_{params["output_suffix"]}.nii.gz')
            result_img.to_filename(output_path)
            print(f"Saved Fisher Z-transformed RDM map for {subject_id} ({condition_name}, {mask_name}) to {output_path}")

            # Save the plot for quality control
            image_output_path = os.path.join(condition_images_dir, f'{subject_id}_rsa_map_{params["output_suffix"]}.png')
            plot_stat_map(result_img, bg_img=template_path, title=f"{subject_id} RSA Toolbox RDM Map ({condition_name}, {mask_name})",
                          display_mode='z', threshold=0.2, cut_coords=10, colorbar=True, output_file=image_output_path)
            print(f"Saved QC image for {subject_id} to {image_output_path}")
